## Import and cleaning the Census data

In [86]:
# Loading neccessary libraries
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
import numpy as np
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
# Import Census data from local machine
sf_tract = pd.read_csv('C:/Datasets/pdb_tract.csv')

In [21]:
sf_tract.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74021 entries, 0 to 74020
Data columns (total 567 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   FIPS_Tract                        int64  
 1   State                             int64  
 2   State_name                        object 
 3   County                            int64  
 4   County_name                       object 
 5   Tract                             int64  
 6   Flag                              float64
 7   Num_BGs_in_Tract                  float64
 8   LAND_AREA                         float64
 9   AIAN_LAND                         float64
 10  URBANIZED_AREA_POP_CEN_2010       float64
 11  URBAN_CLUSTER_POP_CEN_2010        float64
 12  RURAL_POP_CEN_2010                float64
 13  Tot_Population_CEN_2010           float64
 14  Tot_Population_ACS_09_13          float64
 15  Tot_Population_ACSMOE_09_13       float64
 16  Males_CEN_2010                    float

In [22]:
# Dropping columns cotain MOE as it is the margin of error which not needed for prediction now
sf_tract = sf_tract.drop(list(sf_tract.filter(regex='MOE')), axis=1)

In [23]:
# We also drop administrative data columns such as state name, county name, ... which are not useful for prediction
sf_tract = sf_tract.drop(['FIPS_Tract', 'State', 'State_name', 'County', 'County_name', 'Tract', 'Flag', 'Num_BGs_in_Tract'], axis=1)

In [29]:
# Filter a list of currency columns need to convert
currency_columns = list(sf_tract.select_dtypes(include='object').columns)

In [30]:
# Finally, convert currency to numeric value
for col in currency_columns:
    sf_tract[col] = pd.to_numeric(sf_tract[col].str.replace('$','').str.replace(',',''))

In [32]:
# Check that we have all as numeric values
sf_tract.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74021 entries, 0 to 74020
Data columns (total 328 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   LAND_AREA                         float64
 1   AIAN_LAND                         float64
 2   URBANIZED_AREA_POP_CEN_2010       float64
 3   URBAN_CLUSTER_POP_CEN_2010        float64
 4   RURAL_POP_CEN_2010                float64
 5   Tot_Population_CEN_2010           float64
 6   Tot_Population_ACS_09_13          float64
 7   Males_CEN_2010                    float64
 8   Males_ACS_09_13                   float64
 9   Females_CEN_2010                  float64
 10  Females_ACS_09_13                 float64
 11  Pop_under_5_CEN_2010              float64
 12  Pop_under_5_ACS_09_13             float64
 13  Pop_5_17_CEN_2010                 float64
 14  Pop_5_17_ACS_09_13                float64
 15  Pop_18_24_CEN_2010                float64
 16  Pop_18_24_ACS_09_13               float

## Features selection through variance and correlation

In [46]:
# Normalize the variance and select only high variance columns above 0.1
sel = VarianceThreshold(threshold=0.1)

sel.fit(sf_tract/sf_tract.mean())

mask = sel.get_support()

reduced_sf = sf_tract.loc[:, mask]

print(reduced_sf.shape)

(74021, 294)


We able to delete 34 low variance features but we still have almost 300 features to simplify more

In [48]:
# Remove highly correlated features
# Create positive correlation matrix
corr_sf = reduced_sf.corr().abs()

# Create a mask matrix
mask = np.triu(np.ones_like(corr_sf, dtype=bool))

In [49]:
# Apply the mask to our correlation matrix
tri_sf = corr_sf.mask(mask)

tri_sf

,LAND_AREA,AIAN_LAND,URBANIZED_AREA_POP_CEN_2010,URBAN_CLUSTER_POP_CEN_2010,RURAL_POP_CEN_2010,Tot_Population_CEN_2010,Tot_Population_ACS_09_13,Males_CEN_2010,Males_ACS_09_13,Females_CEN_2010,...,pct_No_Plumb_ACS_09_13,pct_Recent_Built_HU_ACS_09_13,avg_Agg_House_Value_ACS_09_13,pct_TEA_Update_Leave_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010,has_superfund
LAND_AREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIAN_LAND,0.138494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
URBANIZED_AREA_POP_CEN_2010,0.097833,0.115712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
URBAN_CLUSTER_POP_CEN_2010,0.000938,0.083958,0.395151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RURAL_POP_CEN_2010,0.126386,0.134151,0.508525,0.052665,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pct_Deletes_CEN_2010,0.232142,0.093368,0.186264,0.026366,0.200614,0.064649,0.072610,0.048771,0.059225,0.078448,...,0.221844,0.018899,0.058307,0.557772,0.417008,NaN,NaN,NaN,NaN,NaN
pct_Census_UAA_CEN_2010,0.013447,0.010064,0.242361,0.065845,0.149966,0.150913,0.150741,0.133859,0.135862,0.163503,...,0.293767,0.041144,0.186868,0.241292,0.118135,0.168444,NaN,NaN,NaN,NaN
pct_RPLCMNT_FRMS_CEN_2010,0.074072,0.006048,0.011521,0.010686,0.104436,0.073131,0.073484,0.068739,0.071325,0.075435,...,0.123624,0.019189,0.195700,0.218925,0.049749,0.142718,0.292236,NaN,NaN,NaN
pct_BILQ_Mailout_count_CEN_2010,0.033593,0.041169,0.160551,0.033900,0.150169,0.066488,0.065078,0.079626,0.076216,0.051787,...,0.000939,0.027021,0.127203,0.060406,0.022173,0.037464,0.035702,0.137980,NaN,NaN


In [51]:
# Find columns that meet threshold
to_drop = [c for c in tri_sf.columns if any(tri_sf[c] > 0.7)]

# Drop those columns
reduced_sf = reduced_sf.drop(to_drop, axis=1)

reduced_sf.shape

(74021, 93)

We see that we are able to drop from 294 columns to 93 columns that have correlation with each other below 0.7

In [54]:
# We have imbalance classes with missing values only in one majority class so it is safe to delete those missing rows
# Further, will still have 93% of original dataset after dropping these rows rather than imputing
reduced_sf = reduced_sf.dropna(how='any')

reduced_sf.shape

(68776, 93)

## Dealing with imbalanced classes

In [55]:
# Seperate input features and target
X = reduced_sf.drop('has_superfund', axis=1)
y = reduced_sf['has_superfund']

In [61]:
# Summarize class distribution
counter = Counter(y)

print(counter)

Counter({0: 67633, 1: 1143})


#### Decision tree evaluated on imbalanced dataset

In [67]:
# Define model
model = DecisionTreeClassifier()
# Evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.505


We see that the classification perfom just like a random guess at 50% on this extreme imbalanced dataset

#### Borderline-SMOTE

In [71]:
# Transform the dataset
oversample = BorderlineSMOTE()
X, y = oversample.fit_resample(X, y)
# Summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({0: 67633, 1: 67633})


In [72]:
# Define model
model = DecisionTreeClassifier()
# Evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.971


#### Borderline-SMOTE SVM

In [73]:
# Re-seperate input features and target
X = reduced_sf.drop('has_superfund', axis=1)
y = reduced_sf['has_superfund']

# Summarize class distribution
counter = Counter(y)
print(counter)

Counter({0: 67633, 1: 1143})


In [74]:
# transform the dataset
oversample = SVMSMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({0: 67633, 1: 34232})


In [75]:
# Define model
model = DecisionTreeClassifier()
# Evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.962


#### Adaptive Synthetic Sampling (ADASYN)

In [76]:
# Re-seperate input features and target
X = reduced_sf.drop('has_superfund', axis=1)
y = reduced_sf['has_superfund']

# Summarize class distribution
counter = Counter(y)
print(counter)

Counter({0: 67633, 1: 1143})


In [79]:
# transform the dataset
oversample = ADASYN()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({0: 67633, 1: 67275})


In [80]:
# Define model
model = DecisionTreeClassifier()
# Evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.951


Among three different methods of resampling the dataset, Borderline-SMOTE gave us the best performance so we will resample the Census dataset using this method

#### Create a balanced class dataset

In [81]:
# Re-seperate input features and target
X = reduced_sf.drop('has_superfund', axis=1)
y = reduced_sf['has_superfund']

# Transform the dataset
oversample = BorderlineSMOTE()
X, y = oversample.fit_resample(X, y)

Mean ROC AUC: 0.971


In [82]:
# Save a copy of this full X,y
X1 = X
y1 = y

## Selecting features for model performance

In [87]:
# Pre-processing the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)

#### Creating a logistic regression model

In [91]:
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

X_test_std = scaler.transform(X_test)

y_pred = lr.predict(X_test_std)

print(accuracy_score(y_test, y_pred))

0.6703548546081813


#### Recursive Feature Elimination with Logistic Regression

In [92]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=10)

rfe.fit(X_train_std, y_train)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=10, step=1, verbose=0)

In [94]:
# Inspecting the RFE results
X.columns[rfe.support_]

Index(['Age5p_Italian_ACS_09_13', 'Age5p_OthEuro_ACS_09_13',
       'Civ_unemp_45_64_ACS_09_13', 'pct_URBAN_CLUSTER_POP_CEN_2010',
       'pct_Age5p_Italian_ACS_09_13', 'pct_Age5p_Yiddish_ACS_09_13',
       'pct_Age5p_OthEuro_ACS_09_13', 'pct_Civ_unemp_45_64_ACS_09_13',
       'pct_Diff_HU_1yr_Ago_ACS_09_13', 'avg_Agg_House_Value_ACS_09_13'],
      dtype='object')

In [95]:
print(accuracy_score(y_test, rfe.predict(X_test_std)))

0.6436914736323311


#### Random forest classifier

In [96]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

print(accuracy_score(y_test, rf.predict(X_test)))

0.9907836372597338


#### RFE with Random Forest

In [100]:
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=10)

rfe.fit(X_train, y_train)

print(X.columns[rfe.support_])

Index(['LAND_AREA', 'FRST_FRMS_CEN_2010', 'pct_RURAL_POP_CEN_2010',
       'pct_Non_Inst_GQ_CEN_2010', 'pct_NH_AIAN_alone_ACS_09_13',
       'pct_NH_SOR_alone_CEN_2010', 'pct_MLT_U10p_ACS_09_13',
       'pct_Mobile_Homes_ACS_09_13', 'pct_No_Plumb_ACS_09_13',
       'avg_Agg_House_Value_ACS_09_13'],
      dtype='object')


In [101]:
print(accuracy_score(y_test, rfe.predict(X_test)))

0.9831690487925087


We see that by using only 10 features out of 567 original feautures, we can predict the status of Superfund with 98% accuracy on a balanced class dataset. If we want to achieve 99% accuracy then we need to use a total of 93 features out of 567 original features.